In [9]:
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('linear').getOrCreate()
# Load training data
all_data = spark.read.format("libsvm").load("sample_libsvm_data.txt")
trainin, test = all_data.randomSplit([0.7, 0.3])

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(training)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

# We can also use the multinomial family for binary classification
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")

# Fit the model
mlrModel = mlr.fit(training)

# Print the coefficients and intercepts for logistic regression with multinomial family
print("Multinomial coefficients: " + str(mlrModel.coefficientMatrix))
print("Multinomial intercepts: " + str(mlrModel.interceptVector))

24/05/03 17:30:27 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


Coefficients: (692,[272,300,323,350,351,378,379,405,406,407,428,433,434,435,455,456,461,462,483,484,489,490,496,511,512,517,539,540,568],[-7.520689871384e-05,-8.11577314684689e-05,3.8146927718465075e-05,0.0003776490540424333,0.00034051483661944016,0.0005514455157343123,0.0004085386116096918,0.00041974673327494546,0.0008119171358670042,0.000502770837266876,-2.3929260406599642e-05,0.0005745048020902312,0.0009037546426803624,7.818229700243747e-05,-2.178755195291058e-05,-3.4021658217894325e-05,0.0004966517360637645,0.0008190557828370383,-8.017982139522497e-05,-2.7431694037834025e-05,0.00048108322262389945,0.00048408017626778825,-8.92647292000764e-06,-0.0003414881233042727,-8.95059257412124e-05,0.00048645469116892205,-8.478698005185958e-05,-0.00042347832158317646,-7.296535777631108e-05])
Intercept: -0.5991460286401453
Multinomial coefficients: 2 X 692 CSRMatrix
(0,272) 0.0001
(0,300) 0.0001
(0,350) -0.0002
(0,351) -0.0001
(0,378) -0.0003
(0,379) -0.0002
(0,405) -0.0002
(0,406) -0.0004
(0,40

In [11]:
training.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                100|
|   mean|               0.57|
| stddev|0.49756985195624287|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+



In [16]:
result = lrModel.evaluate(test)

In [19]:
#result.rootMeanSquaredError
prd_test = test.select(['features'])

In [20]:
pred = lrModel.transform(prd_test)

In [31]:
from pyspark.sql.window import Window
from pyspark.sql.functions import monotonically_increasing_id, row_number
w = Window.orderBy(monotonically_increasing_id())

pred =  pred.withColumn("columnindex", row_number().over(w))
n_test =  test.select('label').withColumn("columnindex", row_number().over(w))

combined_result = pred.join(n_test, pred.columnindex == n_test.columnindex, 'inner').drop(n_test.columnindex)

In [32]:
combined_result.show()

24/05/03 17:52:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/03 17:52:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/03 17:52:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/03 17:52:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/03 17:52:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/03 17:52:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/03 1

+--------------------+--------------------+--------------------+----------+-----------+-----+
|            features|       rawPrediction|         probability|prediction|columnindex|label|
+--------------------+--------------------+--------------------+----------+-----------+-----+
|(692,[95,96,97,12...|[0.90244229778955...|[0.71145113687931...|       0.0|          1|  0.0|
|(692,[98,99,100,1...|[0.91570187077986...|[0.71416552338942...|       0.0|          2|  0.0|
|(692,[100,101,102...|[0.59914602864014...|[0.64546090674371...|       0.0|          3|  0.0|
|(692,[123,124,125...|[0.86550215722893...|[0.70380892937235...|       0.0|          4|  0.0|
|(692,[124,125,126...|[0.87504721062690...|[0.70579483114752...|       0.0|          5|  0.0|
|(692,[126,127,128...|[0.88666346545209...|[0.70820114937300...|       0.0|          6|  0.0|
|(692,[126,127,128...|[0.90462805375380...|[0.71189963979263...|       0.0|          7|  0.0|
|(692,[126,127,128...|[0.77915443052624...|[0.68549784575086